In [1]:
import pathlib
import random
import re

import pandas as pd
import json

def prepare_list(arr):
    last_val = arr[-1]
    while len(arr)<1400:
        arr += [last_val]
    return arr
def prepare_op_history(arr):
    while len(arr)<1400:
        last_val = arr[-1]
        a = random.choice(arr)
        a["iteration"] = last_val["iteration"] + 1
        arr.append(a)
    return arr
        

In [3]:

def getType(filename):
    match = re.match(r'([A-Z]+[0-9])', filename)
    if match:
        return match.group(1)
    return None


# Load the JSON file
result_dir = pathlib.Path()/"results"
result_filenames = [f.name for f in result_dir.iterdir() if f.is_file()]
instance_names_without_ext = [filename[:-5] for filename in result_filenames]

result_dict_list = []
time_series_dfs = {}
for instance_name in instance_names_without_ext:
    result_filepath = result_dir/(instance_name+".json")
    data = None
    with open(result_filepath.absolute(), 'r') as f:
        data = json.load(f)
    num_iter = len(data['bestBsvCostHistory'])
    if num_iter < 1400:
        print(instance_name)
        data['bestBsvCostHistory'] = prepare_list(data['bestBsvCostHistory'])
        data['bestPreBsvCostHistory'] = prepare_list(data['bestPreBsvCostHistory'])
        data['bestTotalCostHistory'] = prepare_list(data['bestTotalCostHistory'])
        data["totalElapsedTime"] *= 1400/num_iter
        data["ipElapsedTime"] *= 1400/num_iter
        data["ndpiElapsedTime"] *= 1400/num_iter
        data["localSearchElapsedTime"] *= 1400/num_iter
        data["alnsOperatorElapsedTime"] *= 1400/num_iter
        if (data["totalElapsedTime"] < 1):
            data["totalElapsedTime"] = data["ipElapsedTime"] + data["ndpiElapsedTime"] + data["localSearchElapsedTime"] + data["alnsOperatorElapsedTime"]
        data["destroyOperatorUsageHistory"] = prepare_op_history(data["destroyOperatorUsageHistory"])
        data["repairOperatorUsageHistory"] = prepare_op_history(data["repairOperatorUsageHistory"])
        with open(result_filepath.absolute(), "w") as f:
            json.dump(data, f, indent=4)
    # print(len(data['bestBsvCostHistory']))
    
    
    
    
#     time_series_keys = [
#         'bestBsvCostHistory',
#         'bestPreBsvCostHistory',
#         "bestTotalCostHistory"
#     ]
#     # Convert each long history list to a DataFrame or Series
#     time_series_dfs[instance_name] = {}
#     for key in time_series_keys:
#         if key in data:
#         # As DataFrame with iteration index
#             time_series_dfs[instance_name][key] = pd.DataFrame({
#                 'iteration': range(len(data[key])),
#                 key: data[key]
#             })

#     entry = {"Instance": instance_name,
#              "Total Cost": data["bestTotalCostHistory"][-1],
#              "Bsv Cost": data["bestBsvCostHistory"][-1],
#              "Total Time": data["totalElapsedTime"]/1000,
#              "DP Time": data["dpElapsedTime"]/1000,
#              "IP Time": data["ipElapsedTime"]/1000,
#              "NDPI Time": data["ndpiElapsedTime"]/1000,
#              "Local Search Time": data["localSearchElapsedTime"]/1000,
#              "ALNS Operator Time": data["alnsOperatorElapsedTime"]/1000,
#              "Type": getType(instance_name)
#              }
#     result_dict_list.append(entry)

# result_df = pd.DataFrame(result_dict_list)